<a href="https://colab.research.google.com/github/NULabTMN/homework-2-AkshatSavaliya/blob/main/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank

In this assignment, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs (NOT the power method with explicit matrix multiplication).

Consider the following directed graph:

![A directed link graph](https://ccs.neu.edu/home/dasmith/courses/cs6200/pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [1]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F'),
               (6, 'G'),
               (7,'H')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [2]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4),
  (5, 6),
  (4, 6)
]


xx = 1

We use integer identifiers for the nodes for efficiency. Note that, unlike this example, in a real web graph, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [3]:
# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.


outlinks = {}

for node in small_nodes:
  if node[0] not in outlinks:
    outlinks[node[0]] = []

for item in small_edges:
  outlinks[item[0]].append(item[1])

print(outlinks)


def page_rank_fixed_iter(nodes, edges, iterations=10):
  P = len(nodes)
  I = [0] * P
  R = [0] * P

  for i in range(P):
    I[i] = 1 / P
  
  for i in range(iterations):
    for i in range(P):
      R[i] = 0.15 / P
    
    final_val = 0

    for node in nodes:
      Q = outlinks[node[0]]
      count = 0
      if len(Q):
        for q in Q:
          R[q] += (1 - 0.15) * I[node[0]] / len(Q)
      else:
        count += 1
        # for q in nodes:
        #   R[q[0]] += (1-0.15) * I[node[0]] / P 
    
      
      final_val += count * ((1-0.15) * I[node[0]] / P)
      # for q in nodes:
      #   R[q[0]] += count * ((1-0.15) * I[node[0]] / P)
    
    for q in nodes:
      R[q[0]] += final_val

    for i in range(P):
      I[i] = R[i]
  
  print(sum(R))
  return R
      

# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.
print(page_rank_fixed_iter(small_nodes, small_edges, 1))
print(page_rank_fixed_iter(small_nodes, small_edges, 10))
print(page_rank_fixed_iter(small_nodes, small_edges, 100))

{0: [1, 2, 5], 1: [2, 3, 4, 5], 2: [3, 4], 3: [0, 2, 4, 5], 4: [0, 6], 5: [0, 1, 4, 6], 6: [], 7: []}
0.9999999999999999
[0.1515625, 0.10729166666666667, 0.13385416666666666, 0.125, 0.17812499999999998, 0.13385416666666666, 0.125, 0.0453125]
0.9999999999999999
[0.1625252777388798, 0.11143691705643147, 0.1319584667231671, 0.11710561740876502, 0.1700293876530271, 0.1319584667231671, 0.1376426670894, 0.037343199607162245]
0.9999999999999994
[0.16253046791326042, 0.11143369232324157, 0.1319570611832602, 0.11710392820128275, 0.17002938844549811, 0.1319570611832602, 0.13764588317048784, 0.03734251757970834]


## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [4]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

--2022-03-09 16:24:50--  https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703486 (3.5M) [application/x-gzip]
Saving to: ‘vertices-edu.txt.gz’

vertices-edu.txt.gz 100%[===================>]   3.53M  2.75MB/s    in 1.3s    

2022-03-09 16:24:52 (2.75 MB/s) - ‘vertices-edu.txt.gz’ saved [3703486/3703486]

--2022-03-09 16:24:53--  https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12829526 (12M) [application/x-gzip]
Saving to: ‘edges-edu.txt.gz’

edges-edu.txt.gz    100%[===================>]  12.23M  5.81MB/s    in 2.1s    

2022-03-09 16:24:56 (5.81 MB/s) - ‘edges-edu.txt.gz’ saved [12829526/128295

There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [5]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.
import csv
with open("vertices-edu.txt") as load_file:
    reader = csv.reader(load_file, delimiter=" ")
    m = [tuple(r) for r in reader]
new_node = tuple((int(x[0]), x[1]) for x in m)

import csv
with open("edges-edu.txt") as load_file:
    reader = csv.reader(load_file, delimiter=" ")
    new_edges = [tuple(row) for row in reader]
edges_p = tuple((int(x[0]), int(x[1])) for x in new_edges)


In [6]:
print(new_node[:5])
print(edges_p[:5])

((0, 'edu.00zl5e'), (1, 'edu.06hxbt'), (2, 'edu.082ifc'), (3, 'edu.083mjs'), (4, 'edu.09xzrr'))
((386, 440), (19202, 1033), (103884, 2635), (342306, 7399), (8366, 8312))


Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall that we defined entropy when talking about data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [7]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.

outlinks = {}

for node in new_node:
  if node[0] not in outlinks:
    outlinks[node[0]] = []

for item in edges_p:
  outlinks[item[0]].append(item[1])

import math

def get_perplexity(I):
    hpr = 0
    for page in I:
        hpr += page*(math.log2(page))
    return 2**hpr


def page_rank(nodes, edges, threshold=1):
  P = len(nodes)
  I = [0] * P
  R = [0] * P

  for i in range(P):
    I[i] = 1 / P
  
  for i in range(15):
    for i in range(P):
      R[i] = 0.15 / P
    
    final_val = 0

    for node in nodes:
      Q = outlinks[node[0]]
      count = 0
      if len(Q):
        for q in Q:
          R[q] += (1 - 0.15) * I[node[0]] / len(Q)
      else:
        count += 1 
     
      final_val += count * ((1-0.15) * I[node[0]] / P)

    
    for q in nodes:
      R[q[0]] += final_val

    for i in range(P):
      I[i] = R[i]
    
    print(get_perplexity(I))
  
  print(sum(R))

  return R
      

# Run until perplexity changes by less than 1
PR = page_rank(new_node, edges_p, 1)

3.354900353014811e-06
3.910178281925948e-06
4.268902029324276e-06
4.425249562550732e-06
4.5138779226088696e-06
4.5551038166181605e-06
4.580832680156119e-06
4.594069425897551e-06
4.603245899508705e-06
4.608308020240469e-06
4.612222051557467e-06
4.614493030990435e-06
4.616398779324345e-06
4.6175615052796726e-06
4.618591448145051e-06
0.9999999999981641


## Link Analysis

\[20 points\] In this final section, you will compute some properties of the web-site graph you downloaded.

First, consider the _in-link count_ of a website, simply the number of web-sites pointing to it (including self-links). 

In [12]:
# TODO: List the document ID, domain name, and in-link count of the 60 websites with the highest in-link count

nodes_dict = {}
for item in new_node:
  nodes_dict[item[0]] = item[1]

inlinks = {}
for node in new_node:
  if node[0] not in inlinks:
    inlinks[node[0]] = []

for edge in edges_p:
  inlinks[edge[1]].append(edge[0])

inlinks_count = {}
for item in inlinks.items():
  inlinks_count[item[0]] = len(item[1])

sorted_inlinks =  sorted(inlinks_count.items(), key = lambda kv: kv[1],reverse=True)

xx = 0
for item in sorted_inlinks:
  print("id: ", item[0], " Domain Name: ", nodes_dict[item[0]], " Inlink count: ", item[1])
  xx += 1
  if xx == 20:
    break





id:  185524  Domain Name:  edu.mit.web  Inlink count:  4388
id:  278032  Domain Name:  edu.stanford  Inlink count:  4021
id:  244433  Domain Name:  edu.purdue.english.owl  Inlink count:  3531
id:  140443  Domain Name:  edu.indiana  Inlink count:  3339
id:  237176  Domain Name:  edu.princeton  Inlink count:  3251
id:  64587  Domain Name:  edu.columbia  Inlink count:  3123
id:  465503  Domain Name:  edu.yale  Inlink count:  2804
id:  418623  Domain Name:  edu.utexas  Inlink count:  2622
id:  383763  Domain Name:  edu.unc  Inlink count:  2592
id:  197698  Domain Name:  edu.nap  Inlink count:  2494
id:  439637  Domain Name:  edu.washington  Inlink count:  2291
id:  373442  Domain Name:  edu.umich  Inlink count:  2281
id:  440674  Domain Name:  edu.washington.depts  Inlink count:  2276
id:  148945  Domain Name:  edu.jhu.muse  Inlink count:  2255
id:  60975  Domain Name:  edu.colorado  Inlink count:  2232
id:  449738  Domain Name:  edu.wisc  Inlink count:  2230
id:  38320  Domain Name:  edu.

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [14]:
# TODO: List the document ID, domain name, and PageRank of the 60 websites with the highest PageRank.

PR_dict = {}
for i, rank in enumerate(PR):
  PR_dict[i] = rank

sorted_PR =  sorted(PR_dict.items(), key = lambda kv: kv[1],reverse=True)

xx = 0
for item in sorted_PR:
  print("id: ", item[0], " Domain Name: ", nodes_dict[item[0]], " Inlink count: ", item[1])
  xx += 1
  if xx == 20:
    break




id:  185524  Domain Name:  edu.mit.web  Inlink count:  0.0009287435339940903
id:  465503  Domain Name:  edu.yale  Inlink count:  0.0006992387033356765
id:  318278  Domain Name:  edu.ucdavis.cas  Inlink count:  0.0005984715790444966
id:  278032  Domain Name:  edu.stanford  Inlink count:  0.0005949124171482952
id:  136464  Domain Name:  edu.illinois  Inlink count:  0.0005799358302359056
id:  237176  Domain Name:  edu.princeton  Inlink count:  0.0005022964971967422
id:  319795  Domain Name:  edu.ucdavis.ice.code  Inlink count:  0.0004943562734855353
id:  284517  Domain Name:  edu.stanford.web  Inlink count:  0.0004751176157734379
id:  383763  Domain Name:  edu.unc  Inlink count:  0.0004721288382615311
id:  64587  Domain Name:  edu.columbia  Inlink count:  0.00044770361757707794
id:  449738  Domain Name:  edu.wisc  Inlink count:  0.0004363423554472278
id:  60975  Domain Name:  edu.colorado  Inlink count:  0.0004326714113083783
id:  346969  Domain Name:  edu.ucsf  Inlink count:  0.000430011

Finally, compute some summary statistics on this dataset.

In [21]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);

no_inlink_counts = 0
for item in inlinks.items():
  if len(item[1]) == 0:
    no_inlink_counts += 1


print("proportion of websites with no in-links: ", no_inlink_counts / len(new_node))


# - the proportion of websites with no out-links (i.e., sink nodes);

no_outlink_counts = 0
for item in outlinks.items():
  if len(item[1]) == 0:
    no_outlink_counts += 1


print("proportion of websites with no in-links: ", no_outlink_counts / len(new_node))


# - the proportion of websites whose PageRank is higher than the initial uniform distribution.

initial_distribution_prob = 1 / len(new_node) 
print(initial_distribution_prob)

count = 0
for rank in PR:
  if rank > initial_distribution_prob:
    count += 1

print("The proportion of websites whose PageRank is higher than the initial uniform distribution: ", count / len(PR) )

proportion of websites with no in-links:  0.26153633041013563
proportion of websites with no in-links:  0.6106641661427643
2.1311283258921438e-06
The proportion of websites whose PageRank is higher than the initial uniform distribution:  0.15525696079789444
